# Bot 配置系统调试笔记本

本笔记本用于调试和测试 Bot 配置系统：
- YAML 配置文件加载
- Bot 配置解析
- 多 Bot 平台管理

In [ ]:
# 添加项目根目录到路径
import sys
import os
sys.path.insert(0, os.path.dirname(os.getcwd()))

# 设置必要的环境变量
os.environ.setdefault("TELEGRAM_BOT_TOKEN", "test_token")
os.environ.setdefault("OPENAI_API_KEY", "test_key")

## 1. 导入 Bot 配置模块

In [ ]:
from src.bot.config_loader import (
    BotConfigLoader,
    BotConfig,
    AIConfig,
    PromptConfig,
    RoutingConfig,
    LimitsConfig,
    AgentsConfig,
    MessagesConfig,
    get_config_loader
)

print("✅ Bot 配置模块导入成功")

## 2. 加载 Bot 配置

In [ ]:
# 创建配置加载器
loader = BotConfigLoader(bots_dir="../bots")

# 列出所有可用的 Bot
available_bots = loader.list_bots()

print(f"发现 {len(available_bots)} 个可用的 Bot:")
for bot_id in available_bots:
    print(f"  - {bot_id}")

In [ ]:
# 加载所有 Bot 配置
all_configs = loader.load_all_configs()

print(f"加载了 {len(all_configs)} 个 Bot 配置:")
for bot_id, config in all_configs.items():
    print(f"\n=== {bot_id} ===")
    print(f"  名称: {config.name}")
    print(f"  描述: {config.description}")
    print(f"  类型: {config.bot_type}")
    print(f"  语言: {config.language}")

## 3. 查看 SoulmateBot 配置详情

In [ ]:
# 加载 soulmate_bot 配置
soulmate_config = loader.load_config("soulmate_bot")

if soulmate_config:
    print("SoulmateBot 配置详情:")
    print("=" * 50)
    
    # 基础信息
    print(f"\n📝 基础信息:")
    print(f"  名称: {soulmate_config.name}")
    print(f"  描述: {soulmate_config.description}")
    print(f"  用户名: {soulmate_config.username}")
    print(f"  类型: {soulmate_config.bot_type}")
    print(f"  语言: {soulmate_config.language}")
    print(f"  是否公开: {soulmate_config.is_public}")
else:
    print("❌ 未找到 soulmate_bot 配置")

In [ ]:
# AI 配置
if soulmate_config:
    ai = soulmate_config.ai
    
    print(f"\n🤖 AI 配置:")
    print(f"  Provider: {ai.provider}")
    print(f"  模型: {ai.model}")
    print(f"  温度: {ai.temperature}")
    print(f"  最大 Token: {ai.max_tokens}")
    print(f"  上下文窗口: {ai.context_window}")

In [ ]:
# 路由配置
if soulmate_config:
    routing = soulmate_config.routing
    
    print(f"\n🔀 路由配置:")
    print(f"  模式: {routing.mode}")
    print(f"  私聊自动回复: {routing.private_chat_auto_reply}")
    print(f"  群聊需要@: {routing.group_chat_mention_required}")

In [ ]:
# 限额配置
if soulmate_config:
    limits = soulmate_config.limits
    
    print(f"\n📊 限额配置:")
    print(f"  免费版:")
    print(f"    消息: {limits.free_tier.messages}")
    print(f"    图片: {limits.free_tier.images}")
    print(f"  基础版:")
    print(f"    消息: {limits.basic_tier.messages}")
    print(f"    图片: {limits.basic_tier.images}")
    print(f"  高级版:")
    print(f"    消息: {limits.premium_tier.messages}")
    print(f"    图片: {limits.premium_tier.images}")
    
    # 使用便捷方法获取限额
    print(f"\n  使用 get_limit 方法:")
    print(f"    免费版消息限额: {soulmate_config.get_limit('free', 'messages')}")
    print(f"    高级版消息限额: {soulmate_config.get_limit('premium', 'messages')}")

In [ ]:
# 消息模板
if soulmate_config:
    messages = soulmate_config.messages
    
    print(f"\n💬 消息模板:")
    print(f"\n  欢迎消息:")
    print(f"    {messages.welcome}")
    print(f"\n  帮助消息:")
    print(f"    {messages.help}")

In [ ]:
# 获取系统提示词
if soulmate_config:
    system_prompt = soulmate_config.get_system_prompt()
    
    print(f"\n🎯 系统提示词:")
    print("-" * 40)
    print(system_prompt)

## 4. 查看 AssistantBot 配置

In [ ]:
# 加载 assistant_bot 配置
assistant_config = loader.load_config("assistant_bot")

if assistant_config:
    print("AssistantBot 配置详情:")
    print("=" * 50)
    print(f"\n📝 基础信息:")
    print(f"  名称: {assistant_config.name}")
    print(f"  描述: {assistant_config.description}")
    print(f"  类型: {assistant_config.bot_type}")
    
    print(f"\n🤖 AI 配置:")
    print(f"  Provider: {assistant_config.ai.provider}")
    print(f"  模型: {assistant_config.ai.model}")
    
    print(f"\n📊 限额配置:")
    print(f"  免费版消息: {assistant_config.get_limit('free', 'messages')}")
    print(f"  高级版消息: {assistant_config.get_limit('premium', 'messages')}")
else:
    print("❌ 未找到 assistant_bot 配置")

## 5. 功能检查

In [ ]:
# 检查功能是否启用
if soulmate_config:
    features_to_check = [
        "emotional_support",
        "daily_companion",
        "code_execution",
        "web_search",
        "conversation_memory"
    ]
    
    print("SoulmateBot 功能状态:")
    for feature in features_to_check:
        enabled = soulmate_config.is_feature_enabled(feature)
        status = "✅ 启用" if enabled else "❌ 禁用"
        print(f"  {feature}: {status}")

## 6. 配置热重载测试

In [ ]:
# 测试配置重载
print("测试配置重载:")

# 第一次加载
config1 = loader.get_config("soulmate_bot")
print(f"  首次加载: {config1.name if config1 else 'None'}")

# 重载配置
config2 = loader.reload_config("soulmate_bot")
print(f"  重载后: {config2.name if config2 else 'None'}")

# 验证是不同的对象
print(f"  是否为不同对象: {config1 is not config2}")

## 7. 创建自定义 Bot 配置示例

In [ ]:
# 手动创建 Bot 配置对象（不通过 YAML）
custom_config = BotConfig(
    name="CustomBot",
    description="自定义测试机器人",
    username="custom_bot",
    bot_type="assistant",
    language="zh",
    ai=AIConfig(
        provider="openai",
        model="gpt-4o-mini",
        temperature=0.8,
        max_tokens=500
    ),
    routing=RoutingConfig(
        mode="auto",
        private_chat_auto_reply=True
    ),
    features_enabled=["chat", "translation"],
    features_disabled=["code_execution"]
)

print("自定义配置:")
print(f"  名称: {custom_config.name}")
print(f"  模型: {custom_config.ai.model}")
print(f"  chat 功能: {'启用' if custom_config.is_feature_enabled('chat') else '禁用'}")
print(f"  code_execution 功能: {'启用' if custom_config.is_feature_enabled('code_execution') else '禁用'}")